In [11]:
import numpy as np
import pandas as pd
import random

In [12]:
def preprocess(data):
    data.text = data.text.apply(lambda x: x.lower())
    data.text = data.text.replace(r'\s+|\\n', ' ', regex=True)
    return data

In [13]:
train_data = pd.read_csv("train_data.csv")
train_data = preprocess(train_data)
print(train_data.head())

test_data = pd.read_csv("test_data.csv")
test_data = preprocess(test_data)
print(test_data.head())


  language                                               text    label
0    dansk   dette er et fremragende initiativ, og jeg stø...  Ireland
1    dansk   hr. formand, jeg er sikker på, at alle her er...  Ireland
2    dansk   hr. formand, folk på den nordlige halvkugle t...  England
3    dansk   hr. formand, med forbehold af nogle få ændrin...  England
4    dansk   - hr. formand, jeg må protestere mod den lemf...  England
                                                text
0   hr. formand, selv om vi i høj grad sympatiser...
1   quiero dejar constancia de mi apoyo a este in...
2   . – el comercio ilegal de riñones humanos se ...
3   signor presidente, per introdurre una nota di...
4   jeg stemte for meddelelsen af decharge til fæ...


In [14]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
mappings = []
for col in ["language", "label"]:
    train_data[col] = le.fit_transform(train_data[col])
    mappings.append(dict(zip(le.transform(le.classes_), le.classes_)))

print(mappings)

[{0: 'Deutsch', 1: 'Nederlands', 2: 'dansk', 3: 'español', 4: 'italiano'}, {0: 'England', 1: 'Ireland', 2: 'Scotland'}]


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

count_vec = CountVectorizer()
train_bow = count_vec.fit_transform(train_data.text)
test_bow = count_vec.transform(test_data.text)

model = MultinomialNB().fit(train_bow, train_data.label)
predicted_labels = model.predict(test_bow)
print(predicted_labels)

[0 1 1 ... 0 0 0]


In [16]:
final_data = {"id": test_data.index+1, "label": predicted_labels}

submission = pd.DataFrame(data=final_data).set_index("id")
submission = submission.label.apply(lambda x: mappings[1][x])
submission.to_csv("submission.csv")
submission.head()

id
1     England
2     Ireland
3     Ireland
4     England
5    Scotland
Name: label, dtype: object